#### Install and import dependencies

In [ ]:
!pip install -r requirements.txt

In [ ]:
# Check for spark dependency (ONLY for LOCAL SPARK INSTALLATION)

import findspark
findspark.init()

In [ ]:
#Import PySpark dependencies
from pyspark.sql.functions import split, col, regexp_replace, trim

import pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
import paramiko
import yaml
import os
import shutil
import gzip
import pandas as pd
import csv

In [ ]:
# Widen the display of Pandas DataFrame output
pd.set_option('display.max_colwidth', None)

#### Read SFTP credentials from YAML file

In [ ]:
sftp_cred = yaml.load(open('sftp_credentials.yml'), yaml.FullLoader)
hostname = sftp_cred['hostname']
username = sftp_cred['username']

#### Establish Connection to SFTP Server

In [ ]:
k = paramiko.RSAKey.from_private_key_file("./sftp_ssh.pem")
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

print("Connecting to host...")
ssh.connect(hostname = hostname, username = username, pkey = k)

print("Authenticated.\nStarting the session...")
sftp = ssh.open_sftp()
print("Connected.")

In [ ]:
# Get list of files in the sftp directory
files = sftp.listdir()
print(str(len(files)) + " Files to be processed:")
print(files)

In [ ]:
# Truncate and create a folders to store raw and curated data 
if os.path.exists('./tmp/'):
    shutil.rmtree('./tmp/')
    os.mkdir('./tmp/')
    os.mkdir('./tmp/raw/')
    os.mkdir('./tmp/curated/')

In [ ]:
def transform_data(filename):
    '''Takes in filename to be processed
  
    Transforms the data as follows:
    - Updates column datatypes
    - Column enrichments
    
    Loads data to '/tmp/curated/' local folder 
  
    Parameters:
    filename (string): Name of the file
  
    '''
    
    #Read CSV
    df = spark.read.option("header", "true")\
                .option("quote", "\"")\
                .option("multiline","true")\
                .csv("./tmp/raw/" + filename, header=True)
    
    #Set SparkSession time zone as UTC
    spark.conf.set('spark.sql.session.timeZone', 'UTC')
    
    #Convert string attributes to timestamp
    df = df.withColumn("STORY_DATE_TIME",df['STORY_DATE_TIME']\
                       .cast('timestamp'))
    df = df.withColumn("TAKE_DATE_TIME",df['TAKE_DATE_TIME']\
                       .cast('timestamp'))
    
    #Remove '\n\r' special characters from the text attributes
    df = df.withColumn("ACCUMULATED_STORY_TEXT", regexp_replace(col("ACCUMULATED_STORY_TEXT"), "[\n\r]", " "))
    df = df.withColumn("TAKE_TEXT", regexp_replace(col("TAKE_TEXT"), "[\n\r]", " "))
    
    #Extract Keywords from the text  
    df = df.withColumn('ACCUMULATED_STORY_TEXT_KEYWORDS', trim(split(df['ACCUMULATED_STORY_TEXT'], 'Keywords:')\
                   .getItem(1)))
    df = df.withColumn('TAKE_TEXT_KEYWORDS', trim(split(df['TAKE_TEXT'], 'Keywords:')\
                   .getItem(1)))
    
    #Reorder columns in the data set
    df = df.select("DATE","TIME","UNIQUE_STORY_INDEX","EVENT_TYPE", "PNAC", "STORY_DATE_TIME", "TAKE_DATE_TIME",\
                   "HEADLINE_ALERT_TEXT", "ACCUMULATED_STORY_TEXT", "ACCUMULATED_STORY_TEXT_KEYWORDS", "TAKE_TEXT",\
                   "TAKE_TEXT_KEYWORDS", "PRODUCTS", "RELATED_RICS", "NAMED_ITEMS", "HEADLINE_SUBTYPE", "STORY_TYPE",\
                   "TABULAR_FLAG", "ATTRIBUTION", "LANGUAGE")
    
    #Export data set to csv format
    df.toPandas().to_csv("./tmp/curated/" + filename[:-3], quoting=csv.QUOTE_ALL)

In [ ]:
#Fetch files from SFTP Server
for f in files[0:1]:
    print("Downloading " + f + " ...")
    sftp.get(f, "./tmp/raw/" + f)
    print("Raw data downloaded.\nTransforming data...")
    transform_data(f)
    print("Curated data loaded.")

In [ ]:
df = spark.read.option("header", "true")\
                .option("quote", "\"")\
                .option("multiline","true")\
                .csv("./tmp/raw/2013-07-01.csv.gz", header=True)
    
    #Set SparkSession time zone as UTC
spark.conf.set('spark.sql.session.timeZone', 'UTC')
    
    #Convert string attributes to timestamp
df = df.withColumn("STORY_DATE_TIME",df['STORY_DATE_TIME']\
                   .cast('timestamp'))
df = df.withColumn("TAKE_DATE_TIME",df['TAKE_DATE_TIME']\
                   .cast('timestamp'))

#Remove '\n\r' special characters from the text attributes
df = df.withColumn("ACCUMULATED_STORY_TEXT", regexp_replace(col("ACCUMULATED_STORY_TEXT"), "[\n\r]", " "))
df = df.withColumn("TAKE_TEXT", regexp_replace(col("TAKE_TEXT"), "[\n\r]", " "))

#Extract Keywords from the text  
df = df.withColumn('ACCUMULATED_STORY_TEXT_KEYWORDS', trim(split(df['ACCUMULATED_STORY_TEXT'], 'Keywords:')\
               .getItem(1)))
df = df.withColumn('TAKE_TEXT_KEYWORDS', trim(split(df['TAKE_TEXT'], 'Keywords:')\
               .getItem(1)))

#Reorder columns in the data set
df = df.select("DATE","TIME","UNIQUE_STORY_INDEX","EVENT_TYPE", "PNAC", "STORY_DATE_TIME", "TAKE_DATE_TIME",\
               "HEADLINE_ALERT_TEXT", "ACCUMULATED_STORY_TEXT", "ACCUMULATED_STORY_TEXT_KEYWORDS", "TAKE_TEXT",\
               "TAKE_TEXT_KEYWORDS", "PRODUCTS", "RELATED_RICS", "NAMED_ITEMS", "HEADLINE_SUBTYPE", "STORY_TYPE",\
               "TABULAR_FLAG", "ATTRIBUTION", "LANGUAGE")

#Export data set to csv format
#df.toPandas().to_csv("./tmp/curated/" + filename[:-3], quoting=csv.QUOTE_ALL)

### Future Scope

- Column enrichments like extracing Page Editor, Reporting by, Editing by, etc. (from "ACCUMULATED_STORY_TEXT" or "TAKE_TEXT" attribute) which would simplify querying and searching through the data.
- Few attributes have records containing articles and tables, these could be stored in document databases.
- The data contains text from multiple languages - Arabic, Spanish, etc.; these could be translated using APIs and standardized to English.